#Pendahuluan
Kita akan belajar bagaimana melakukan web scraping untuk mengumpulkan informasi pekerjaan dari website pencarian kerja, khususnya Jobstreet. Kita akan memanfaatkan library seperti Requests dan BeautifulSoup untuk mengambil dan memproses data dari website tersebut, lalu menyimpan hasilnya ke dalam format CSV.

Scraping yang akan kita lakukan mencakup beberapa halaman dari hasil pencarian pekerjaan pada website Jobstreet. Kita akan menyimpan detail seperti posisi pekerjaan, perusahaan, lokasi, gaji, dan waktu posting pekerjaan.


#Prasyarat
Sebelum memulai, pastikan kita memiliki pengetahuan dasar mengenai:

- Python Programming
- HTTP Request dan Response
- HTML dan elemen-elemen dasar seperti div, span, dll.
- Pandas untuk pengolahan data

#Mengimpor Library yang Diperlukan
Pertama-tama, kita perlu mengimpor library yang akan kita gunakan, yaitu requests untuk mengirimkan permintaan HTTP, BeautifulSoup dari library bs4 untuk parsing HTML, dan pandas untuk memproses serta menyimpan data.

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

- Requests: Digunakan untuk mengirim permintaan HTTP dan mengambil konten halaman web.
- BeautifulSoup: Untuk memproses HTML dari halaman web agar lebih mudah diambil datanya.
- Pandas: Untuk menyimpan data yang telah kita scrape ke dalam format tabel (DataFrame) dan kemudian menyimpan ke CSV.

#Menyediakan URL dan Header
Kita mulai dengan mendefinisikan URL dari halaman yang ingin kita scrape. Pada contoh ini, kita akan mengambil data dari halaman pertama hasil pencarian pekerjaan Data Scientist di Jobstreet.

In [2]:
url = 'https://id.jobstreet.com/id/data-scientist-jobs?sortmode=ListedDate'

Selanjutnya, kita perlu menyediakan header untuk permintaan HTTP. Hal ini diperlukan agar permintaan kita terlihat seperti berasal dari browser sungguhan dan bukan dari bot.

In [3]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.6668.90"
}

- User-Agent: Ini adalah informasi tentang browser yang digunakan. Dengan menyediakan User-Agent, kita memberitahu server bahwa permintaan berasal dari browser tertentu (Google Chrome, dalam hal ini).

#Mengirim Permintaan HTTP
Setelah URL dan header sudah siap, kita dapat mengirim permintaan HTTP ke website menggunakan requests.get() untuk mengambil konten halaman tersebut.

In [4]:
s = requests.session()
s.headers.update(headers)

page = s.get(url)

- Session: Kita menggunakan session untuk mengelola header dan request secara lebih efisien.
- get(): Mengirim permintaan HTTP ke URL yang telah ditentukan.

#Memeriksa Status Permintaan (Request Status)
Sangat penting untuk memeriksa apakah permintaan kita berhasil atau tidak. Status respons HTTP akan memberi kita informasi apakah data berhasil diambil.

In [5]:
if page.status_code == 200:
    print("Request berhasil!")
else:
    print(f"Request gagal dengan kode status: {page.status_code}")

Request berhasil!


- status_code: Memberikan informasi status dari HTTP response. Jika statusnya 200, berarti permintaan berhasil. Jika bukan 200, bisa jadi ada masalah, seperti 404 Not Found atau 403 Forbidden.

Tanda permintaan berhasil adalah ketika page.status_code == 200. Jika kode status lain muncul, seperti 404 (halaman tidak ditemukan) atau 403 (akses ditolak), maka kita harus memeriksa ulang URL atau menyesuaikan request.

#Memproses HTML dengan BeautifulSoup
Jika permintaan berhasil, kita dapat melanjutkan dengan mengambil konten dari halaman tersebut dan memprosesnya menggunakan BeautifulSoup.

In [6]:
soup = BeautifulSoup(page.text, "lxml")
joblist = soup.find('div', class_='papho0 _21bfxf1')

- BeautifulSoup: Mengubah konten HTML menjadi objek yang bisa kita proses.
- find(): Mencari elemen HTML berdasarkan tag dan class. Di sini, kita mencari div yang berisi daftar pekerjaan.

#Mengambil Data dari Setiap Pekerjaan
Setelah kita menemukan daftar pekerjaan, langkah selanjutnya adalah mengambil detail dari setiap pekerjaan yang ada pada halaman tersebut.

In [ ]:
jobs_data = []
for artikel in joblist.find_all('article', {'class': 'papho0 papho1 _1j97a3y7i _1j97a3y6e _1j97a3y9q _1j97a3y8m _1j97a3yh _1j97a3y66 _1j97a3y5e vrt92ib vrt92i9 vrt92ia w75d4w18 w75d4w1b _1j97a3y32 _1j97a3y35'}):
    posisi = artikel.find('div', {'class': 'papho0 _1j97a3y5g _1j97a3y52'})
    perusahaan = artikel.find(attrs={'data-automation':'jobCompany'})
    lokasi = artikel.find(attrs={'data-automation':'jobLocation'})
    gaji = artikel.find('span', {'class':'papho0 _15904gq2 _1j97a3y4y _1j97a3y0 _1j97a3yr _15904gq4'})
    waktu = artikel.find('span',{'class':'papho0 _1j97a3y4y w75d4w0 w75d4w1 w75d4w22 _1708b944 w75d4w7'})

- find_all(): Mencari semua elemen article yang sesuai dengan class yang diberikan.
- posisi, perusahaan, lokasi, gaji, dan waktu: Mencari detail pekerjaan dari setiap elemen.

##Menangani Elemen yang Tidak Ditemukan
Jika elemen tertentu tidak ditemukan, kita harus memastikan program tetap berjalan dengan memberikan nilai default. Kita menggunakan operator ternary untuk melakukan ini.

In [ ]:
posisi_text = posisi.get_text() if posisi else "Posisi tidak ditemukan"
perusahaan_text = perusahaan.get_text() if perusahaan else "Perusahaan tidak ditemukan"
lokasi_text = lokasi.get_text() if lokasi else "Lokasi tidak ditemukan"
gaji_text = gaji.get_text() if gaji else "Gaji tidak disebutkan"
waktu_text = waktu.get_text() if waktu else "Waktu tidak ditemukan"

- get_text(): Mengambil teks dari elemen HTML. Jika elemen tidak ditemukan, kita memberikan nilai default seperti "Posisi tidak ditemukan".

## Menyimpan Data ke List
Setelah semua data berhasil diambil, kita menyimpan data tersebut dalam sebuah list jobs_data.

In [ ]:
jobs_data.append({
    "Posisi": posisi_text,
    "Perusahaan": perusahaan_text,
    "Lokasi": lokasi_text,
    "Gaji": gaji_text,
    "Waktu": waktu_text
})

- append(): Menambahkan dictionary berisi data pekerjaan ke dalam list jobs_data

## Kode lengkap pengambilan data

In [7]:
jobs_data = []

for artikel in joblist.find_all('article', {'class': 'papho0 papho1 _1j97a3y7i _1j97a3y6e _1j97a3y9q _1j97a3y8m _1j97a3yh _1j97a3y66 _1j97a3y5e vrt92ib vrt92i9 vrt92ia w75d4w18 w75d4w1b _1j97a3y32 _1j97a3y35'}):
    posisi = artikel.find('div', {'class': 'papho0 _1j97a3y5g _1j97a3y52'})
    perusahaan = artikel.find(attrs={'data-automation':'jobCompany'})
    lokasi = artikel.find(attrs={'data-automation':'jobLocation'})
    gaji = artikel.find('span', {'class':'papho0 _15904gq2 _1j97a3y4y _1j97a3y0 _1j97a3yr _15904gq4'})
    waktu = artikel.find('span',{'class':'papho0 _1j97a3y4y w75d4w0 w75d4w1 w75d4w22 _1708b944 w75d4w7'})

    posisi_text = posisi.get_text() if posisi else "Posisi tidak ditemukan"
    perusahaan_text = perusahaan.get_text() if perusahaan else "Perusahaan tidak ditemukan"
    lokasi_text = lokasi.get_text() if lokasi else "Lokasi tidak ditemukan"
    gaji_text = gaji.get_text() if gaji else "Gaji tidak disebutkan"
    waktu_text = waktu.get_text() if waktu else "Waktu tidak ditemukan"

    print(posisi_text)
    print(perusahaan_text)
    print(lokasi_text)
    print(gaji_text)
    print(waktu_text)
    print("==========================================")

    jobs_data.append({
        "Posisi": posisi_text,
        "Perusahaan": perusahaan_text,
        "Lokasi": lokasi_text,
        "Gaji": gaji_text,
        "Waktu": waktu_text
    })

Data Analyst
PT Gunung Sari Indonesia
Medan
Rp 3.700.000 – Rp 4.000.000 per month
17 menit yang lalu
Data Analis
PT Barberking Indonesia
Sukoharjo
Gaji tidak disebutkan
3 jam yang lalu
Data Engineer
PT ZURICH ASURANSI INDONESIA, Tbk
Jakarta Selatan
Gaji tidak disebutkan
19 jam yang lalu
AI Engineer - SaaS (WFO from Bali)
PT Michael Page Internasional Indonesia
Denpasar
0
21 jam yang lalu
Data Enterprise Analyst
PT Mandiri Tunas Finance
Jakarta Pusat
Gaji tidak disebutkan
21 jam yang lalu
Research & Data analyst
Hara Anargya
Bandung
Gaji tidak disebutkan
1 hari yang lalu
Data Analyst
WINOD RITEL INDONESIA
Jakarta Barat
Gaji tidak disebutkan
1 hari yang lalu
Data Engineer
PT Esha Parama Teknologi
Jakarta Raya
Gaji tidak disebutkan
1 hari yang lalu
Technical Consultant Analyst
PT. Metrodata Electronics, Tbk
Kalideres
Gaji tidak disebutkan
1 hari yang lalu
18. DATA ANALYST
PT Yamaha Motor Electronics Indonesia
Semarang
Gaji tidak disebutkan
1 hari yang lalu
Data Engineer
Devoteam
Kebayoran

#Menyimpan Data dari Satu Halaman ke CSV
Setelah data dari satu halaman terkumpul, kita bisa menyimpan hasilnya ke dalam file CSV.

In [8]:
df_jobs = pd.DataFrame(jobs_data)
df_jobs.to_csv('[Jobstreet]Data_Job_Listings.csv', index=False)

- pd.DataFrame(): Mengubah list jobs_data menjadi DataFrame Pandas.
- to_csv(): Menyimpan DataFrame ke dalam file CSV

In [11]:
df_jobs.head()

,Posisi,Perusahaan,Lokasi,Gaji,Waktu
0,Data Analyst,PT Gunung Sari Indonesia,Medan,Rp 3.700.000 – Rp 4.000.000 per month,17 menit yang lalu
1,Data Analis,PT Barberking Indonesia,Sukoharjo,Gaji tidak disebutkan,3 jam yang lalu
2,Data Engineer,"PT ZURICH ASURANSI INDONESIA, Tbk",Jakarta Selatan,Gaji tidak disebutkan,19 jam yang lalu
3,AI Engineer - SaaS (WFO from Bali),PT Michael Page Internasional Indonesia,Denpasar,0,21 jam yang lalu
4,Data Enterprise Analyst,PT Mandiri Tunas Finance,Jakarta Pusat,Gaji tidak disebutkan,21 jam yang lalu


#Scraping Multi Halaman
Setelah berhasil scraping satu halaman, kita bisa memperluas scraping ke beberapa halaman sekaligus. Kita bisa menggunakan loop untuk mengambil data dari halaman 1 hingga halaman tertentu.

In [12]:
jobs_data = []

for i in range(1,8):
  url = f'https://id.jobstreet.com/id/Data-Scientist-jobs?page={i}&sortmode=ListedDate'

  headers = {
      "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.6668.90"
    }

  s = requests.session()
  s.headers.update(headers)

  try:
      page = s.get(url)
      soup = BeautifulSoup(page.text, "lxml")
      joblist = soup.find('div', class_='papho0 _21bfxf1')

      for artikel in joblist.find_all('article', {'class': 'papho0 papho1 _1j97a3y7i _1j97a3y6e _1j97a3y9q _1j97a3y8m _1j97a3yh _1j97a3y66 _1j97a3y5e vrt92ib vrt92i9 vrt92ia w75d4w18 w75d4w1b _1j97a3y32 _1j97a3y35'}):
          posisi = artikel.find('div', {'class' : 'papho0 _1j97a3y5g _1j97a3y52'})
          perusahaan = artikel.find(attrs={'data-automation':'jobCompany'})
          lokasi = artikel.find(attrs={'data-automation':'jobLocation'})
          gaji = artikel.find('span', {'class':'papho0 _15904gq2 _1j97a3y4y _1j97a3y0 _1j97a3yr _15904gq4'})
          waktu = artikel.find('span',{'class':'papho0 _1j97a3y4y w75d4w0 w75d4w1 w75d4w22 _1708b944 w75d4w7'})

          posisi_text = posisi.get_text() if posisi else "Posisi tidak ditemukan"
          perusahaan_text = perusahaan.get_text() if perusahaan else "Perusahaan tidak disebutkan"
          lokasi_text = lokasi.get_text() if lokasi else "Lokasi tidak ditemukan"
          gaji_text = gaji.get_text() if gaji else "Gaji tidak disebutkan"
          waktu_text = waktu.get_text() if waktu else "Waktu tidak ditemukan"

          jobs_data.append({
            "Posisi": posisi_text,
            "Perusahaan": perusahaan_text,
            "Lokasi": lokasi_text,
            "Gaji": gaji_text,
            "Waktu": waktu_text
          })

          print(posisi_text)
          # print(perusahaan_text)
          # print(lokasi_text)
          # print(gaji_text)
          # print(waktu_text)
          print("===============================")
  except requests.exceptions.RequestException as e:
      print(f"Error: {e}")

Data Analyst
Data Analis
Data Engineer
AI Engineer - SaaS (WFO from Bali)
Data Enterprise Analyst
Research & Data analyst
Data Analyst
Data Engineer
Technical Consultant Analyst
18. DATA ANALYST
Data Engineer
Machine Learning Engineer
Data Analyst - Snr II
Commercial Operations Intern
Product Analyst
Senior Data Analyst
R&D Technical Service Officer
Data Management Coordinator
Data Scientist
Data Analyst
Senior Machine Learning Engineer
Artificial Intelligence Researcher
Data Analyst Staff
Data Analyst
Data Analyst
Senior Data Analyst Marketing
Marketing Insight Intern
Big Data Operation
Senior Data Scientist - Hybrid Set up (Semarang)
Data Engineer - Data Warehouse
Data Analyst
Data Analyst & Automation Specialist Penempatan di Gatot Subroto
Data Analyst
Data Processing Executive
Data Engineer Internship
Data Analyst
Machine Learning Developer (Mid-Senior Level)
Project Data Management
Data Analyst
Analytical Assistant Scientist
Data Engineer
Data Analyst
Data Analyst
Data Engineer - 

In [13]:
df_jobs_full = pd.DataFrame(jobs_data)

df_jobs_full.to_csv('[Jobstreet]Full_Data_Job_Listings.csv', index=False)

In [14]:
df_jobs_full.head()

,Posisi,Perusahaan,Lokasi,Gaji,Waktu
0,Data Analyst,PT Gunung Sari Indonesia,Medan,Rp 3.700.000 – Rp 4.000.000 per month,2 jam yang lalu
1,Data Analis,PT Barberking Indonesia,Sukoharjo,Gaji tidak disebutkan,5 jam yang lalu
2,Data Engineer,"PT ZURICH ASURANSI INDONESIA, Tbk",Jakarta Selatan,Gaji tidak disebutkan,21 jam yang lalu
3,AI Engineer - SaaS (WFO from Bali),PT Michael Page Internasional Indonesia,Denpasar,0,22 jam yang lalu
4,Data Enterprise Analyst,PT Mandiri Tunas Finance,Jakarta Pusat,Gaji tidak disebutkan,23 jam yang lalu


In [15]:
df_jobs_full.shape

(220, 5)